KNN
- K-Nearest-Neighbors
    - K개의 가까운 이웃
- 1) 새로운 관측값(목표 변수) 입력
- 2) 기존 데이터중 가장 속성이 비슷한 k개의 이웃을 찾음
- 3) 목표값과 같은 값으로 분류하여 예측

In [19]:
# <예제7-4> KNN 분류 알고리즘

import pandas as pd
import seaborn as sns

'''
step 1 데이터 준비
'''

df = sns.load_dataset("titanic")

# Ipython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option("display.max_columns",15)
print(df.head())

print("\n")

'''
step 2 데이터 탐색
'''
print(df.info())
print("\n")

# "deck"열(누락 데이터)와 "embarked"열(중복 데이터) 제거
rdf = df.drop(["deck","embark_town"],axis = 1)
print(rdf.head())
print('\n')

# "age"열의 누락데이터 갖는 행 제거
rdf = rdf.dropna(subset = ["age"],how = "any",axis =0)
print("새로운 행의 개수:",len(rdf))

# "embarked"열의 NaN을 최빈값으로 치환
most_freq = rdf["embarked"].value_counts(dropna = True).idxmax()
print(most_freq)
print("\n")
rdf["embarked"].fillna(most_freq,inplace= True)

'''
step 3 속성 선택
예측변수 - 결과값(목표)
설명 변수 - 예측을 위해 모형이 사용하는 속성
'''

# 분석에 활용할 열(속성) 선택
ndf = rdf[["survived","pclass","sex","age","sibsp","parch","embarked"]]
print(ndf.head())

print("\n")

# "sex"열 과 "embarked"열의 범주형 데이터를 숫자형으로 변환
# 원핫인 코딩 - 더미 변수를 만드는 과정
onehot_sex = pd.get_dummies(ndf["sex"])
ndf =pd.concat([ndf,onehot_sex], axis = 1)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = "town")
ndf = pd.concat([ndf,onehot_embarked], axis = 1)
# 기존 "sex"열과 "embarked"열 삭제
ndf.drop(["sex","embarked"],axis = 1, inplace= True)
print(ndf.head())
print("\n")

'''
step 4 데이터셋 구분 - 훈련용(train data) / 검증용(test data)
1) 예측변수(survived) - y변수에 저장
   설명변수(나머지 열) - X변수에 저장
2) 설명변수의 값 정규화
3) train_test_split() 메소드로 훈련용/검증용 나눔
'''
#1)
y = ndf["survived"]
X = ndf[["pclass","age","sibsp","parch","female","male","town_C","town_Q","town_S"]]
#2)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
#3)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,
                                                   test_size = 0.3,
                                                   random_state =10)
print("train data개수:",len(X_train))
print("test data개수:", len(X_test))
print("\n")

'''
step 5 모형 학습 및 검증 
- sklearn라이브러리의 neighbors모듈사용
    - KNeibhorsClassification() 함수로 모형 객체 생성
        - n_neighbors = n 으로 n개의 이웃 설정
- 모형 객체(knn).fit(train data) 로 모형 학습
- 모형 객체(knn).predict(X_test)로 결과값예측(y_hat)
'''

from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성
knn = KNeighborsClassifier(n_neighbors = 5)
# train data로 모형 학습
knn.fit(X_train,y_train)
#  X_test로 결과값예측(y_hat)
y_hat = knn.predict(X_test)

# 예측 결과와 길제 결과값 출력
print(y_hat[0:10])
print(y_test.values[0:10])
print("\n")

# 모형 예측 능력 평가
# Confusion Matrix 계산
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test, y_hat)
print(knn_matrix)
print("\n")

# 평가  지표 계산
knn_report = metrics.classification_report(y_test,y_hat)
print(knn_report)

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-